In [ ]:
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical

def prepare_data():
    """データを用意する
    
    Returns:
    X_train(ndarray):
        訓練データ(50000.32.32.3)
    X_test(ndarray):
        テストデータ(10000.32.32.3)
    y_train(ndarray):
        訓練データのOne-Hot化した正解ラベル(50000,10)
    y_train(ndarray):
        テストデータのOne-Hot化した正解ラベル10000,10)
    y_test_label(ndarray):
        テストデータの正解ラベル(10000)
    """
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    
    # 訓練用とテスト用の画像データを標準化する
    # 4次元テンソルのすべての軸方向に対して平均、標準偏差を求めるので
    # axis=(0,1,2,3)は省略してもよい
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train,axis=(0,1,2,3))
    # 標準化する際に分母の標準偏差に極小値を加える
    x_train = (X_train-mean)/(std+1e-7)
    x_test = (X_test-mean)/(std+1e-7)
    
    # テストデータの正解ラベルを2階テンソルから1階テンソルへフラット化
    y_test_label = np.ravel(y_test)
    # 訓練データとテストデータの正解ラベルをOne-Hot表現に変換(10クラス化)
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    
    return X_train, X_test, y_train, y_test, y_test_label 

In [ ]:
from keras.layers import Input, Conv2D, Dense, Activation
from keras.layers import AveragePooling2D, GlobalAvgPool2D
from keras.layers import BatchNormalization
from keras import regularizers
from keras.models import Model

"""basic_conv_block1()
   basic_conv_block2()
   畳み込み層を生成する
   
   Parameters: inp(Input): 入力層
               fsize(int): フィルターのサイズ
               layers(int) : 層の数
   Returns:
     Conv2Dを格納したTensorオブジェクト
"""
def basic_conv_block1(inp, fsize, layers):
    x = inp
    for i in range(layers):
        x = Conv2D(
            filters=fsize,
            kernel_size=3,
            padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
    return x

def basic_conv_block2(inp, fsize, layers):
    weight_decay = 1e-4 # ハイパーパラメーターの値
    x = inp
    for i in range(layers):
        x = Conv2D(
            filters=fsize,
            kernel_size=3,
            padding='same',
            kernel_regularizer=regularizers.l2(weight_decay)
            )(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    return x

def create_cnn(model_num):
    """モデルを生成する

    Parameters: model_num(int):
      モデルの番号
    Returns:
      Conv2Dを格納したModelオブジェクト
    """
    inp = Input(shape=(32,32,3))
    if model_num < 5:
        x = basic_conv_block1(inp, 64, 3)
        x = AveragePooling2D(2)(x)
        x = basic_conv_block1(x, 128, 3)
        x = AveragePooling2D(2)(x)
        x = basic_conv_block1(x, 256, 3)
        x = GlobalAvgPool2D()(x)
        x = Dense(10, activation='softmax')(x)
        model = Model(inp, x)
    else:
        x = basic_conv_block2(inp, 64, 3)
        x = AveragePooling2D(2)(x)
        x = basic_conv_block2(x, 128, 3)
        x = AveragePooling2D(2)(x)
        x = basic_conv_block2(x, 256, 3)
        x = GlobalAvgPool2D()(x)
        x = Dense(10, activation='softmax')(x)
        model = Model(inp, x)
    return model

In [ ]:
import numpy as np

def ensemble_average(models, X):
    """確率の平均をとるアンサンブル
    
    Parameters:
        models(list): Modelオブジェクトのリスト
        X(array): 検証用のデータ
    Returns:
        各画像の正解ラベルを格納した(,10000)のndarray
    """
    # 検証結果のNumpy配列を格納する変数
    preds_sum = None
    # modelsから更新をフリーズされたモデルを取り出す
    for model in models:
        if preds_sum is None:
            # 1番目のモデルが推定した各クラスの確率を代入
            # preds_sumの形状は(データ数,クラス数)
            preds_sum = model.predict(X)
        else:
            # 2番目のモデル以降は推定確率を各クラスごとに加算する
            preds_sum += model.predict(X)
    # 各クラスの推定確率の平均を(データ数,クラス数)の形状で取得
    probs = preds_sum / len(models)
    # 推定確率の平均(データ数,クラス数)の各行(axis=1)から
    # 最大値のインデックスを取得して(,データ数)の形状で返す
    return np.argmax(probs, axis=1)

In [ ]:
from tensorflow.keras.callbacks import Callback

class Checkpoint(Callback):
    """Callbackのサブクラス
    
    Attributes:
        model(object): Modelオブジェクト
        filepath(str): 重みを保存するファイルのパス
        best_val_acc
    """
    def __init__(self, model, filepath):
        """
        Parameters:
            model(Model): 現在実行中のModelオブジェクト
            filepath(str): 重みを保存するファイルのパス
            best_val_acc(int): 1モデルの最も高い精度を保持
        """
        self.model = model
        self.filepath = filepath
        self.best_val_acc = 0.0

    def on_epoch_end(self, epoch, logs):
        """エポック終了時に呼ばれるメソッドをオーバーライド
        
        これまでのエポックより精度が高い場合は重みをファイルに保存する
        
        Parameters:
            epoch(int): エポックの回数
            logs(dict): {'val_acc':損失, 'val_acc':精度}
        """
        if self.best_val_acc < logs['val_acc']:
            # 前回のエポックより精度が高い場合は重みを保存する
            self.model.save_weights(self.filepath)  # ファイルパス
            # 精度をlogsに保存
            self.best_val_acc = logs['val_acc']
            # 重みが保存されたことを精度と共に通知する
            print('Weights saved.', self.best_val_acc)

In [ ]:
import math
import pickle
import numpy as np
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import History

def train(X_train, X_test, y_train, y_test, y_test_label):
    """学習を行う
    
    Parameters:
        X_train(ndarray): 訓練データ
        X_test(ndarray): 訓練データの正解ラベル
        y_train(ndarray): テストデータ
        y_test(ndarray): テストデータの正解ラベル(One-Hot表)
        y_test_label(ndarray): テストデータの正解ラベル
    """
    n_estimators = 9  # アンサンブルするモデルの数
    batch_size = 1024 # ミニバッチの数
    epoch = 80        # エポック数   
    models = []       # モデルを格納するリスト
    # 各モデルの学習履歴を保持するdict
    global_hist = {"hists":[], "ensemble_test":[]}
    # 各モデルの推測結果を登録する2階テンソルを0で初期化
    # (データ数, モデル数)
    single_preds = np.zeros((X_test.shape[0], # 行数は画像の枚数と同じ
                             n_estimators))   # 列数はネットワークの数

    # モデルの数だけ繰り返す
    for i in range(n_estimators):
        # 何番目のモデルかを表示
        print('Model',i+1)
        # CNNのモデルを生成,引数はモデルの番号
        train_model = create_cnn(i)
        # モデルをコンパイルする
        train_model.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=["acc"])
        # コンパイル後のモデルをリストに追加
        models.append(train_model)

        # コールバックに登録するHistoryオブジェクトを生成
        hist = History()
        # コールバックに登録するCheckpointオブジェクトを生成
        cp = Checkpoint(train_model,         # Modelオブジェクト
                        f'weights_{i}.h5') # 重みを保存するファイル名
        # ステップ減衰関数
        def step_decay(epoch):
            initial_lrate = 0.001 # ベースにする学習率
            drop = 0.5            # 減衰率
            epochs_drop = 10.0     # ステップ減衰は10エポックごと
            lrate = initial_lrate * math.pow(
                drop,
                math.floor((1+epoch)/epochs_drop)
            )
            return lrate
            
        lrate = LearningRateScheduler(step_decay) # スケジューラ―オブジェクト
        
        # データ拡張
        datagen = ImageDataGenerator(
            rotation_range=15,      # 15度の範囲でランダムに回転させる
            width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
            height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
            horizontal_flip=True,   # 水平方向にランダムに反転、左右の入れ替え
            zoom_range=0.2,         # ランダムに拡大
            )

        # 学習を行う
        train_model.fit_generator(
            datagen.flow(X_train,
                         y_train,
                         batch_size=batch_size),
            #batch_size=batch_size,
            epochs=epoch,
            steps_per_epoch=X_train.shape[0] // batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[hist, cp, lrate] # コールバック
            )       

        # 学習に用いたモデルで最も精度が高かったときの重みを読み込む
        train_model.load_weights(f'weights_{i}.h5')
        
        # 対象のモデルのすべての重み更新をフリーズする
        for layer in train_model.layers:
            layer.trainable = False

        # テストデータで推測し、各画像ごとにラベルの最大値を求め、
        # 対象のインデックスを正解ラベルとしてsingle_predsのi列に格納
        single_preds[:, i] = np.argmax(train_model.predict(X_test),
                                       axis=-1) # 行ごとの最大値を求める

        # 学習に用いたモデルの学習履歴をglobal_histのhistsキーに登録
        global_hist['hists'].append(hist.history)
        
        # 平均をとるアンサンブルを実行
        ensemble_test_pred = ensemble_average(models, X_test)
        
        # scikit-learn.accuracy_score()でアンサンブルによる精度を取得
        ensemble_test_acc = accuracy_score(y_test_label, ensemble_test_pred)
        
        # アンサンブルの精度をglobal_histのensemble_testキーに追加
        global_hist['ensemble_test'].append(ensemble_test_acc)
        # 現在のアンサンブルの精度を出力
        print('Current Ensemble Test Accuracy : ', ensemble_test_acc)

    global_hist['corrcoef'] = np.corrcoef(single_preds,
                                          rowvar=False) # 列ごとの相関係数を求める
    print('Correlation predicted value')
    print(global_hist['corrcoef'])

In [ ]:
# 実行部

# データを用意する
X_train, X_test, y_train, y_test, y_test_label  = prepare_data()

# アンサンブルを実行
train(X_train, X_test, y_train, y_test, y_test_label)